# Import Libraries

In [9]:
# Standard libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 300)

# for audio files
import librosa

In [10]:
from google.colab import drive
import os
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load training data csv file

In [11]:
df = pd.read_csv('/content/drive/MyDrive/207/207-Project/data/train_metadata.csv')

df.head()

,primary_label,common_name,scientific_name,species_rank,filename,dataset,duration_secs,duration_mins,duration_hrs,species_total_files,...,rating,latitude,longitude,author,url,license,class_weight_file,class_weight_duration,class_weight_combined,orig_index
0,thrnig1,Thrush Nightingale,Luscinia luscinia,0,thrnig1/XC110335.ogg,train,119.275102,1.987918,0.033132,499,...,4.5,58.5264,13.8637,Patrik Åberg,https://www.xeno-canto.org/110335,Creative Commons Attribution-NonCommercial-Sha...,0.937876,0.419757,0.627439,13845
1,thrnig1,Thrush Nightingale,Luscinia luscinia,0,thrnig1/XC110336.ogg,train,122.044127,2.034069,0.033901,499,...,5.0,59.1763,15.4038,Patrik Åberg,https://www.xeno-canto.org/110336,Creative Commons Attribution-NonCommercial-Sha...,0.937876,0.419757,0.627439,13846
2,thrnig1,Thrush Nightingale,Luscinia luscinia,0,thrnig1/XC118260.ogg,train,44.382041,0.739701,0.012328,499,...,5.0,61.5650,29.5650,Steve Klasan,https://www.xeno-canto.org/118260,Creative Commons Attribution-NonCommercial-Sha...,0.937876,0.419757,0.627439,13848
3,thrnig1,Thrush Nightingale,Luscinia luscinia,0,thrnig1/XC120947.ogg,train,189.231066,3.153851,0.052564,499,...,4.5,52.4430,21.0940,Lars Lachmann,https://www.xeno-canto.org/120947,Creative Commons Attribution-NonCommercial-Sha...,0.937876,0.419757,0.627439,13849
4,thrnig1,Thrush Nightingale,Luscinia luscinia,0,thrnig1/XC125024.ogg,train,45.312018,0.755200,0.012587,499,...,2.5,54.5770,11.9226,Louis A. Hansen,https://www.xeno-canto.org/125024,Creative Commons Attribution-NonCommercial-Sha...,0.937876,0.419757,0.627439,13850


# Define extract audio function to ensure sample rate and extract audio object in np.array format using librosa

In [12]:
def extract_audio_librosa(filename, desired_sample_rate=16000):
  '''
  Usage: extract the audio object in np.array format from an audio file
  Input:
    filename
  Output:
    audio object in np.array format
  '''
  filepath = '/content/drive/MyDrive/207/207-Project/data/train/audio_files/' + filename
  waveform, sample_rate = librosa.load(filepath, sr=desired_sample_rate)
  if sample_rate != desired_sample_rate:
    waveform = librosa.resample(y=waveform, orig_sr = sample_rate, target_sr = desired_sample_rate)
  return waveform

# Extract file names

In [13]:
filenames = df['filename']
filenames

0       thrnig1/XC110335.ogg
1       thrnig1/XC110336.ogg
2       thrnig1/XC118260.ogg
3       thrnig1/XC120947.ogg
4       thrnig1/XC125024.ogg
                ...         
3273    cohmar1/XC748724.ogg
3274    cohmar1/XC748726.ogg
3275    cohmar1/XC749483.ogg
3276    cohmar1/XC749638.ogg
3277    cohmar1/XC751198.ogg
Name: filename, Length: 3278, dtype: object

# Create top level directory to save the files

In [ ]:
save_dir = '/content/drive/MyDrive/207/207-Project/data/train/librosa_loaded'

In [ ]:
os.makedirs(save_dir, exist_ok=True)

# Create a list to store the file paths

In [14]:
filepaths = []

# Extract the audio files and save to disk

In [15]:
for i, filename in enumerate(filenames):
  audio = extract_audio_librosa(filename)

  dir = filename.split('/')[0]
  filename = filename.split('/')[1]
  filename = filename.replace('.ogg', '.npy')
  save_dir = f'/content/drive/MyDrive/207/207-Project/data/train/librosa_loaded/{dir}/'

  if os.path.exists(save_dir):
    pass
  else:
    os.makedirs(save_dir, exist_ok=True)

  save_filename = os.path.join(save_dir, filename)

  filepath = f'{dir}/{filename}'
  filepaths.append(filepath)

  with open(save_filename, 'w') as file:
    pass

  np.save(save_filename, audio)

  del audio
  del filepath

# Pick an example to ensure the audio object was saved successfully and correctly

In [16]:
example_b4 = extract_audio_librosa(filenames[0])
example_b4

array([-1.9821441e-06, -1.7455233e-05,  1.8845337e-06, ...,
        1.9377280e-06,  1.1900666e-05,  7.0215829e-06], dtype=float32)

In [17]:
example_b4.shape

(1908402,)

In [18]:
example = np.load('/content/drive/MyDrive/207/207-Project/data/train/librosa_loaded/thrnig1/XC110335.npy')
example

array([-1.9821441e-06, -1.7455233e-05,  1.8845337e-06, ...,
        1.9377280e-06,  1.1900666e-05,  7.0215829e-06], dtype=float32)

In [19]:
example.shape

(1908402,)

In [20]:
np.array_equal(example_b4, example)

True

# Pick another example to ensure the audio object was saved successfully and correctly

In [21]:
example_b4 = extract_audio_librosa(filenames[3277])
example_b4

array([ 3.76428216e-06,  1.29341206e-05, -1.47690735e-05, ...,
       -4.14991891e-03, -2.83075124e-03,  1.61276408e-03], dtype=float32)

In [22]:
example_b4.shape

(276256,)

In [23]:
example = np.load('/content/drive/MyDrive/207/207-Project/data/train/librosa_loaded/cohmar1/XC751198.npy')
example

array([ 3.76428216e-06,  1.29341206e-05, -1.47690735e-05, ...,
       -4.14991891e-03, -2.83075124e-03,  1.61276408e-03], dtype=float32)

In [24]:
example.shape

(276256,)

In [25]:
np.array_equal(example_b4, example)

True

# load one example from df and use in a librosa function to confirm the object is usable

In [26]:
example_og = extract_audio_librosa(df['filename'][0])
example_og_rms = librosa.feature.rms(y=example_og)
example_og_rms

array([[7.07731215e-06, 1.08174545e-05, 3.72313603e-04, ...,
        3.65966582e-04, 2.87620729e-04, 2.30783771e-04]], dtype=float32)

In [29]:
example_new = np.load('/content/drive/MyDrive/207/207-Project/data/train/librosa_loaded/thrnig1/XC110335.npy')
example_new_rms = librosa.feature.rms(y=example_new)
example_new_rms

array([[7.07731215e-06, 1.08174545e-05, 3.72313603e-04, ...,
        3.65966582e-04, 2.87620729e-04, 2.30783771e-04]], dtype=float32)

In [30]:
np.array_equal(example_og_rms, example_new_rms)

True

Note that the filepath has already been saved to the metadata.csv in an older version of this file so it has since been deleted.